In [30]:
import pandas as pd
import os

In [34]:
main = os.getcwd() + '../'

dateFile_p = main + '../data/rawDataset/news/dateonly.csv'
labeled_topics_path = main + '../result/modelOutput/document_topics.csv'
TopdocFile_p = main + '../result/modelOutput/labeled_topics.csv'

#### News

In [35]:
df1 = pd.read_csv(dateFile_p)
df2 = pd.read_csv(TopdocFile_p)

print(f"Date File = {len(df1)}, Data File = {len(df2)}")

df_merged = pd.concat([df1, df2], axis=1)
df_merged.drop(['ID', 'Agency'], axis=1, inplace=True)

print(f" Merged data = {len(df_merged)}")

Date File = 1156987, Data File = 1156987
 Merged data = 1156987


##### Economic Filtered
(เลือกข่าวเฉพาะที่มี labeled topic มาใช้)42

In [29]:
import pandas as pd

df = pd.read_csv(TopdocFile_p)
datedf = pd.read_csv(dateFile_p)
labeled_topics_df = pd.read_csv(labeled_topics_path)
labeled_topics = labeled_topics_df ['Topic'].tolist()


# Define a function to find the top 10 topics per row
def find_top_10_topics(row):
    top_10 = row.nlargest(10)
    return pd.Series([top_10.index.tolist(), top_10.values.tolist()], index=['Top_10_Topics', 'Top_10_Values'])

# Apply the function and organize the resulting top topics
top_10_df = docs_df.apply(find_top_10_topics, axis=1)
dynamic_columns = [f'Top_{i+1}' for i in range(10)]  # Adjusted for top 10
top_10_topics_df = pd.DataFrame(top_10_df['Top_10_Topics'].tolist(), columns=dynamic_columns)

# Merge the dates and top topics, then filter by labeled topics
merged_df = pd.concat([dates_df, top_10_topics_df], axis=1)
merged_df['ECON'] = merged_df.apply(lambda row: any(row.get(f'Top_{i}', None) in labeled_topics for i in range(1, 11)), axis=1)
filtered_df = merged_df[['ID', 'Agency', 'Date', 'ECON']]
complete_df = pd.concat([filtered_df, docs_df], axis=1)

# Set non-ECON topics to zero
topic_columns = [col for col in complete_df.columns if col.startswith('Topic_')]
complete_df.loc[complete_df['ECON'] == False, topic_columns] = 0
complete_df.drop(['ECON', 'ID'], axis=1, inplace=True)

# Group by Date and summarize topic distributions
result = complete_df.groupby('Date').sum(numeric_only=True).reset_index()
result['Date'] = pd.to_datetime(result['Date']).dt.date

# Prepare date range for reindexing
start_date, end_date = pd.to_datetime('2015-03-01').date(), pd.to_datetime('2023-12-31').date()
all_dates = pd.date_range(start_date, end_date).date

# Reindex and fill missing dates with zeros
result.set_index('Date', inplace=True)
result = result.reindex(pd.to_datetime(all_dates), fill_value=0)
result.reset_index(inplace=True)
result.rename(columns={'index': 'Date'}, inplace=True)
result['Date'] = result['Date'].dt.date
result = result.round(8)

# Save to CSV
result.to_csv('path/to/output/filtered_news.csv', index=False)

In [30]:
import pandas as pd

df = pd.read_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Mic/filtered_news.csv')

labeled_topics_df = pd.read_csv(labeled_topics_path)
labeled_topics_list = labeled_topics_df ['Topic'].tolist()

selected_columns = ['Date'] + [col for col in df.columns if col in labeled_topics_list]
filtered_df = df[selected_columns]

filtered_df.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Mic/filtered_news_selected.csv', index=False)


In [31]:
import pandas as pd

df = pd.read_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Mic/news.csv')

labeled_topics_df = pd.read_csv(labeled_topics_path)
labeled_topics_list = labeled_topics_df ['Topic'].tolist()

selected_columns = ['Date'] + [col for col in df.columns if col in labeled_topics_list]
filtered_df = df[selected_columns]

filtered_df.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Mic/news_selected.csv', index=False)


In [32]:
print("Finished")

Finished


##### GDELT

In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = 'E:/sp2023stock/TopicModeling/data/GDELT/GDELT2.csv'
gdelt_v1_df = pd.read_csv(csv_file_path)

gdelt_v1_df['Date'] = pd.to_datetime(gdelt_v1_df['Date'], format='%Y%m%d')
# Convert back to a string in the new format
gdelt_v1_df['Date'] = gdelt_v1_df['Date'].dt.strftime('%Y-%m-%d')
gdelt_v1_df.fillna(0, inplace=True)
gdelt_v1_df['Date'] = pd.to_datetime(gdelt_v1_df['Date'])

result_gdelt_v1 = gdelt_v1_df.groupby('Date').sum().reset_index()
result_gdelt_v1['Date'] = pd.to_datetime(result_gdelt_v1['Date'], errors='coerce').dt.date

# Define the date range scope
start_date = pd.to_datetime('2018-01-01').date()  # Use .date() to match the format in 'Date' column
end_date = pd.to_datetime('2023-12-31').date()
all_dates = pd.date_range(start_date, end_date).date  # Convert to datetime.date for consistency

# Convert 'Date' from datetime.date objects to Timestamps for reindexing
result_gdelt_v1['Date'] = pd.to_datetime(result_gdelt_v1['Date'])
result_gdelt_v1.set_index('Date', inplace=True)
result_gdelt_v1 = result_gdelt_v1.reindex(pd.to_datetime(all_dates), fill_value=0)

result_gdelt_v1.reset_index(inplace=True)
result_gdelt_v1.rename(columns={'index': 'Date'}, inplace=True)
# result_gdelt_v1['Date'] = result_gdelt_v1['Date'].dt.date  # Convert Timestamps back to dates
result_gdelt_v1 = result_gdelt_v1.round(7)
result_gdelt_v1.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/gdelt_v2.csv', index=False)
result_gdelt_v1


In [56]:
# Importing the necessary library
import pandas as pd

df = pd.read_csv('E:/sp2023stock/TopicModeling/result/gdelt/GDELT_V2.csv')

# Converting the 'Topic' strings to lists of topics
df['Topic'] = df['Topic'].apply(lambda x: x.split(', '))

# Convert the DataFrame into a dictionary
industry_topic_dict = df.set_index('Industry')['Topic'].to_dict()


In [57]:
import pandas as pd
from io import StringIO

# Read the data into a pandas DataFrame
df = pd.read_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/gdelt_v2.csv')

# Count non-zero values for each column
non_zero_counts = df.apply(lambda x: (x != 0).sum())
